# Azure ML - Jupyter Notebooks - mlflow

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID, workspace_name)

## Jupyter Notebooks mlflow

Esta parte es la haremos con la interfaz gráfica, por lo que hazlo con el `Workspace` que quieras

Hemos visto cómo hacer un entrenamiento básico con un Jupyter Notebook, ahora vamos a ver cómo hacerlo con `mlflow` para poder hacer seguimiento del entrenamiento

En Studio seleccionamos `Notebooks` en la parte izquierda de la interfaz. Vemos que tenemos una zona con las carpetas y otra en la que podemos darle al botón de `+ Files` para crear nuevos archivos, de modo que le damos al botón, seleccionamos `Create new file`, ponemos un nombre, en mo caso pondré `image-classification-mlflow.ipynb`, en `File type` seleccionamos `Notebook` y le damos a `Create`

Vemos que se nos ha creado un Jupyter Notebook

### Seleccionar un `Compute Instance`

Lo primero que tenemos que hacer es seleccionar un `Compute Instance` para ello pinchar en la zona de `Compute` de la parte superior del notebook. Nos aparecerán los `Compute Instance`s que hayamos creado, seleccionamos uno y le damos al botón de `Start`, tardará un poco en arrancar

### Kernel

Una vez hemos levantado una `Compute Instance` tenemos que elegir un kernel, igual que cuando ejecutamos un Jupyter Notebook en local. Para ellos pinchamos en la zona de `Kernel` en la parte superior del notebook y seleccionamos el kernel que queramos. En mo caso voy a seleccionar `Python 3.10 - SDK v2`

Este kernel es un entorno de conda, por lo que podemos crearnos nuevos si queremos. En la zona donde se ven las carpetas hay un botón con formade una terminal, si dejamos el botón encima aparece el texto `Open terminal`. Si le damos se nos abrirá la terminal de esa `Compute Instance`, por lo que podríamos crear nuevos entornos de conda si queremos. Luego podremos seleccionar esos entornos como kernel

En mi caso voy a usar el kernel `Python 3.10 - SDK v2`, abro la terminal e instalo las siguientes librerías

```bash
conda activate azureml_py310_sdkv2
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
pip install mlflow azureml-mlflow
```

### Dónde editar el Jupyter Notebook

También, cuando levantamos el `Compute Instance` se habilita una zona en la parte superior del notebook que pone `Edit in VS Code`. Si le damos se abrirá un menú en el que podemos seleccionar si editarlo en vscode en la web o en local. Haz lo que prefieras

### Entrenamiento

A continuación pondré el notebook que he usado para entrenar el modelo.

#### Librerías

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import Accuracy
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage
import mlflow
from PIL import Image

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Dataset

Descargamos el dataset

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Vemos una muestra del dataset

In [ ]:
from random import randint

idx = randint(0, len(training_data))
sample_img, label = training_data[idx]
sample_img_PIL = ToPILImage()(sample_img)
print(f"Label: {label}")
sample_img_PIL

Label: 0


In [ ]:
sample_img.shape

torch.Size([1, 28, 28])

Vemos la dimensión del dataset

In [ ]:
len_training_data = len(training_data)
len_test_data = len(test_data)
len_training_data, len_test_data

(60000, 10000)

Obtenemos el número de clases

In [ ]:
set_clases = set({})
for i in range(len_training_data):
    _, label = training_data[i]
    set_clases.add(label)
num_classes = len(set_clases)
print(f"Number of classes: {num_classes}, classes: {set_clases}")

Number of classes: 10, classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


Dejamos el código para crear un subset por si te quieres hacer un subset más pequeño

In [ ]:
factor = 1
subset_training_data = torch.utils.data.Subset(training_data, range(0, len_training_data//factor))
subset_test_data = torch.utils.data.Subset(test_data, range(0, len_test_data//factor))

len(subset_training_data), len(subset_test_data)

(60000, 10000)

#### DataLoaders

In [ ]:
BS = 16
train_dataloader = DataLoader(subset_training_data, batch_size=BS, shuffle=True)
test_dataloader = DataLoader(subset_test_data, batch_size=BS, shuffle=True)

len(train_dataloader), len(test_dataloader)

(3750, 625)

#### Modelo

In [ ]:
class ImageClassifier(nn.Module):
    def __init__(self, num_classes):
        self.num_classes = num_classes
        super(ImageClassifier, self).__init__()
        self.imageClassifier = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.LazyLinear(self.num_classes),
        )

    def forward(self, x):
        return self.imageClassifier(x)

Probamos el modelo con una muestra

In [ ]:
sample_batch = next(iter(train_dataloader))
image_batch, label_batch = sample_batch
model = ImageClassifier(num_classes)
output = model(image_batch)
image_batch.shape, output.shape

(torch.Size([16, 1, 28, 28]), torch.Size([16, 10]))

In [ ]:
mlflow.pytorch.autolog()

2024/11/09 09:42:14 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.4.0, but the installed version is 2.5.1. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.


Aquí pon tus valores para `SUSCRIPTION_ID`, `RESOURCE_GROUP` y `WORKSPACE`

In [ ]:
SUSCRIPTION_ID = ""
RESOURCE_GROUP = ""
WORKSPACE = ""
mlflow.set_tracking_uri = f"azureml://francecentral.api.azureml.ms/mlflow/v1.0/subscriptions/{SUSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.MachineLearningServices/workspaces/{WORKSPACE}"

In [ ]:
mlflow.set_experiment(experiment_name="image-classification")

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


<Experiment: artifact_location='', creation_time=1731141371877, experiment_id='ba0ceff2-ddbe-4308-9a4f-6a846a2dd0c7', last_update_time=None, lifecycle_stage='active', name='image-classification', tags={}>

#### Device

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

Como vemos el modelo está en la CPU

#### Funciones de entrenamiento y evaluación

In [ ]:
def train(dataloader, model, loss_fn, metrics_fn, optimizer, epoch):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)
        metric = metrics_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss = loss.item()
            current = batch
            step = batch // 1000 * (epoch + 1)
            mlflow.log_metric("train_loss", f"{loss:.4f}", step=step)
            mlflow.log_metric("train_accuracy", f"{metric:.4f}", step=step)
            print(f"train loss: {loss:.4f}, train accuracy: {metric:.4f} [{current}/{len(dataloader)}]")

In [ ]:
def evaluate(dataloader, model, loss_fn, metrics_fn, epoch):
    num_batches = len(dataloader)
    model.eval()
    eval_loss = 0
    eval_accuracy = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            eval_loss += loss_fn(pred, y).item()
            eval_accuracy += metrics_fn(pred, y).item()

    eval_loss /= num_batches
    eval_accuracy /= num_batches
    mlflow.log_metric("eval_loss", f"{eval_loss:.4f}", step=epoch)
    mlflow.log_metric("eval_accuracy", f"{eval_accuracy:.4f}", step=epoch)

    print(f"eval loss: {eval_loss:.4f}, eval accuracy: {eval_accuracy:.4f}")

#### Entrenamiento

In [ ]:
epochs = 3
LR = 1e-3
loss_fn = nn.CrossEntropyLoss()
metrics_fn = Accuracy(task="multiclass", num_classes=num_classes).to(device)
model = ImageClassifier(num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
from datetime import datetime

name = f"image_classification_{datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}"
with mlflow.start_run(run_name=name) as run:
    params = {
        "epochs": epochs,
        "learning_rate": LR,
        "batch_size": BS,
        "num_classes": num_classes,
        "loss_fn": loss_fn.__class__.__name__,
        "metrics_fn": metrics_fn.__class__.__name__,
        "optimizer": "Adam",
    }
    mlflow.log_params(params)

    with open("model_summary.txt", "w") as f:
        f.write(str(summary(model, input_size=(BS, 1, 28, 28), device=device)))
    mlflow.log_artifact("model_summary.txt")

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, metrics_fn, optimizer, epoch)
        evaluate(test_dataloader, model, loss_fn, metrics_fn, epoch)
    
    mlflow.pytorch.log_model(model, "models", input_example=image_batch.numpy())

Epoch 1
-------------------------------
train loss: 2.3623, train accuracy: 0.0625 [0/3750]
train loss: 0.4929, train accuracy: 0.8750 [100/3750]
train loss: 0.4513, train accuracy: 0.8750 [200/3750]
train loss: 0.6159, train accuracy: 0.7500 [300/3750]
train loss: 0.2675, train accuracy: 0.8750 [400/3750]
train loss: 0.2816, train accuracy: 0.8750 [500/3750]
train loss: 0.2231, train accuracy: 0.9375 [600/3750]
train loss: 0.5147, train accuracy: 0.6875 [700/3750]
train loss: 0.4221, train accuracy: 0.8750 [800/3750]
train loss: 0.1267, train accuracy: 1.0000 [900/3750]
train loss: 0.3912, train accuracy: 0.8750 [1000/3750]
train loss: 1.0029, train accuracy: 0.6250 [1100/3750]
train loss: 0.1412, train accuracy: 1.0000 [1200/3750]
train loss: 0.2537, train accuracy: 0.8125 [1300/3750]
train loss: 0.3991, train accuracy: 0.6875 [1400/3750]
train loss: 0.2171, train accuracy: 0.9375 [1500/3750]
train loss: 0.3148, train accuracy: 0.8750 [1600/3750]
train loss: 0.2962, train accuracy: 0

2024/11/09 09:44:37 INFO mlflow.tracking._tracking_service.client: 🏃 View run image_classification_2024-11-09_09:42:15 at: https://francecentral.api.azureml.ms/mlflow/v2.0/subscriptions/40018f20-6173-46c6-9434-ce9df3089dce/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-gui-/#/experiments/ba0ceff2-ddbe-4308-9a4f-6a846a2dd0c7/runs/15cf21b0-576e-420b-b4f1-aaeef2352ddc.
2024/11/09 09:44:37 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://francecentral.api.azureml.ms/mlflow/v2.0/subscriptions/40018f20-6173-46c6-9434-ce9df3089dce/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-gui-/#/experiments/ba0ceff2-ddbe-4308-9a4f-6a846a2dd0c7.


Si aquí nos vamos a la sección de `Jobs` de nuestro `Workspace` veremos que se ha creado un nuevo `Experiment` con el nombre que le hemos dado. Dentro en la pestaña `Metrics` podremos ver gráfcas las métricas que hemos ido guardando y en la pestaña `Outputs + logs` podremos ver los logs de la ejecución y una carpeta llamada `models` con el modelo guardado del experimento

Lo bueno de mlflow es que tiene integración con la mayoría de librerías de machine learning, por lo que este modelo después puede ser importado a través de mlflow y ser usado en cualquier otro sitio, de hecho vamos a hacerlo

#### Descarga del modelo 

In [ ]:
logged_model = f"runs:/{run.info.run_id}/models"
loaded_model = mlflow.pytorch.load_model(logged_model)

Probamos el modelos entrenado con una muestra del conjunto de test

In [ ]:
try:
    idx = randint(0, len(subset_test_data))
except:
    print(idx, len(subset_test_data))
sample_test_img, label = subset_test_data[idx]
output = loaded_model(sample_test_img.to(output.device).unsqueeze(0))
output.shape

torch.Size([1, 10])

In [ ]:
predicted_label = torch.argmax(output, dim=1).item()
print(f"Predicted label: {predicted_label}, True label: {label}")
sample_test_img_PIL = ToPILImage()(sample_test_img)
sample_test_img_PIL

Predicted label: 5, True label: 5


Como vemos el entrenamiento ha sido exitoso